In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.subplots as sp
import unicodedata
from rapidfuzz import fuzz
import re
from typing import Dict, List, Union, Tuple
import json
from pathlib import Path




# ETL

## Conectar o banco de dados

In [37]:
%run "C:\Users\lopes\Desktop\Portifolio\curriculo-online\src\analysis\db_connection.py"
import sys
from pathlib import Path

project_root = Path().resolve().parent.parent.parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))



c:\Users\lopes\Desktop\Portifolio\curriculo-online\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-05-11 08:28:07,152 - root - INFO - Successfully loaded 1000 rows from BigQuery


Successfully loaded 1000 rows

First 5 rows:
              id           code            product_name  \
0  6111035000430  6111035000430                Sidi Ali   
1  6111242100992  6111242100992                   Perly   
2  6111035002175  6111035002175                Sidi Ali   
3  6111035000058  6111035000058  Eau minérale naturelle   
4  6111252421568  6111252421568                اكوافينا   

                                 brands  \
0                              Sidi Ali   
1                                Jaouda   
2                              sidi ali   
3  Les Eaux Minérales d'oulmès,Sidi Ali   
4                         pepsi,PepsiCo   

                                          categories  \
0  Beverages and beverages preparations,Beverages...   
1  Dairies,Fermented foods,Fermented milk product...   
2  beverages-and-beverages-preparations, beverage...   
3  Beverages and beverages preparations,Beverages...   
4  Boissons et préparations de boissons,Boissons,...   

    

In [38]:
from src.analysis.db_connection import load_data

# Load data
df = load_data()

# Basic inspection
print("\n=== Analise de dados inicial ===")
print("\nPrimeiras 5 linhas:")
print(df.head())

print("\nÚltimas 5 linhas:")
print(df.tail())



c:\Users\lopes\Desktop\Portifolio\curriculo-online\.venv\Lib\site-packages\google\cloud\bigquery\table.py:1933: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
2025-05-11 08:28:10,825 - root - INFO - Successfully loaded 1000 rows from BigQuery



=== Analise de dados inicial ===

Primeiras 5 linhas:
              id           code            product_name  \
0  6111035000430  6111035000430                Sidi Ali   
1  6111242100992  6111242100992                   Perly   
2  6111035002175  6111035002175                Sidi Ali   
3  6111035000058  6111035000058  Eau minérale naturelle   
4  6111252421568  6111252421568                اكوافينا   

                                 brands  \
0                              Sidi Ali   
1                                Jaouda   
2                              sidi ali   
3  Les Eaux Minérales d'oulmès,Sidi Ali   
4                         pepsi,PepsiCo   

                                          categories  \
0  Beverages and beverages preparations,Beverages...   
1  Dairies,Fermented foods,Fermented milk product...   
2  beverages-and-beverages-preparations, beverage...   
3  Beverages and beverages preparations,Beverages...   
4  Boissons et préparations de boissons,Boissons,..

## Limpeza  dos dados

Remover os dados Nulos da  colunas de nomes, categorias e marcas

In [39]:
# Tratamento de dados

df =  df.dropna()

def text_clean(text):
    text = str(text).strip()
    text = " ".join(text.split())
    text = ''.join(
        c for c in unicodedata.normalize('NFKD', text)
        if not unicodedata.combining(c)
    )
    text = text.upper()
    return text

df['product_name'] = df['product_name'].apply(text_clean)
df['categories'] = df['categories'].apply(text_clean)
df['brands'] = df['brands'].apply(text_clean)
print(df.head())

Função para limpar e padronizar os textos das marcas

    Limpas e padronizar o texto da coluna 'brand':
    1. Removendo espaços extras 
    2. Normalizar caracteres usando UNICODE
    3. Converter tudo para maisculo
    4. Remover caracteres especiais
    
    Args:
        text (str): Input de texto para limpas
        
    Returns:
        str: Texto limpo


In [58]:

def clean_brand_text(text: str) -> str:

    if pd.isna(text):
        return ""
    
    # Converte para uma string e remove os caracteres especiais
    text = str(text).strip('[]').strip('"').strip("'").strip()
    
    # Normaliza os caracteres usando UNICODE (NFKD decomposition)
    text = unicodedata.normalize('NFKD', text)
    
    # Remove os acentos
    text = ''.join(c for c in text if not unicodedata.combining(c))
    
    # Converte para maisculo
    text = text.upper()
    
    # Remove caracteres especiais mas mantendo espaços e pontuação
    text = re.sub(r'[^\w\s\-&]', '', text)
    
    # Remove espaços extras
    text = ' '.join(text.split())
    
    return text


Função para padronizar a coluna brand    
    Padronizara listas de marcas que tiver em uma unica string
    
    Args:
        brands (Union[str, List[str]]): Input marcas como string ou lista
        
    Returns:
        str: Padronizar a string da marca

In [ ]:
def standardize_brand_list(brands: Union[str, List[str]]) -> str:

    if isinstance(brands, str):
        brands = [brand.strip() for brand in brands.split(',')]
    return ', '.join(clean_brand_text(brand) for brand in brands)

Importa os dados do dicionario criado em um arquivo json:

    Mapeamento de marcas para padronização
    
    Args:
        mapping (dict): Dicionário de mapeamento de marcas



In [ ]:
def get_brand_mapping() -> Dict[str, str]:

    mapping_file = Path(r"C:\Users\lopes\Desktop\Portifolio\curriculo-online\src\analysis\brand_mapping.json")
    if mapping_file.exists():
        with open(mapping_file, 'r', encoding='utf-8') as f:
            return json.load(f)
    return {}


Procurar marcas similares

In [ ]:
def find_similar_brands(brand: str, existing_brands: List[str], threshold: float = 80.0) -> List[Tuple[str, float]]:

    similar_brands = []
    for existing_brand in existing_brands:
        score = fuzz.ratio(brand, existing_brand)
        if score >= threshold:
            similar_brands.append((existing_brand, score))
    return sorted(similar_brands, key=lambda x: x[1], reverse=True)


Realizar um double check nas marcas para verificar as não mapeadas e achar possiveis marcas.

In [ ]:
def analyze_unmapped_brands(df: pd.DataFrame, mapping: Dict[str, str]) -> pd.DataFrame:

    # Get unique brands from the dataset
    all_brands = df['brands'].unique()
    
    # Find unmapped brands
    unmapped_brands = [brand for brand in all_brands if brand not in mapping]
    
    # Get list of mapped brands (values in the mapping dictionary)
    mapped_brands = list(set(mapping.values()))
    
    # Create results DataFrame
    results = []
    for brand in unmapped_brands:
        similar = find_similar_brands(brand, mapped_brands)
        if similar:
            results.append({
                'unmapped_brand': brand,
                'potential_matches': similar,
                'suggested_mapping': similar[0][0] if similar[0][1] >= 90 else None
            })
    
    return pd.DataFrame(results)

Atualizar o dicionario do arquivo

In [ ]:
def update_brand_mapping(df: pd.DataFrame, auto_accept_threshold: float = 90.0) -> pd.DataFrame:
  
    # Load existing mapping
    mapping = get_brand_mapping()
    
    # Analyze unmapped brands
    unmapped_analysis = analyze_unmapped_brands(df, mapping)
    
    # Auto-accept high-confidence mappings
    high_confidence = unmapped_analysis[
        unmapped_analysis['potential_matches'].apply(
            lambda x: x[0][1] >= auto_accept_threshold if x else False
        )
    ]
    
    # Create new mappings dictionary
    new_mappings = {
        row['unmapped_brand']: row['suggested_mapping']
        for _, row in high_confidence.iterrows()
    }
    
    # Save updated mapping
    if new_mappings:
        save_brand_mapping(new_mappings)
    
    # Return unmapped brands that need manual review
    return unmapped_analysis[
        unmapped_analysis['potential_matches'].apply(
            lambda x: x[0][1] < auto_accept_threshold if x else True
        )
    ]


Salvar o arquivo json do dicionario

In [ ]:
def save_brand_mapping(mapping: Dict[str, str]) -> None:

    mapping_file = Path('data/brand_mapping.json')
    mapping_file.parent.mkdir(parents=True, exist_ok=True)
    
    # Load existing mappings if file exists
    existing_mapping = {}
    if mapping_file.exists():
        with open(mapping_file, 'r', encoding='utf-8') as f:
            existing_mapping = json.load(f)
    
    # Update with new mappings
    existing_mapping.update(mapping)
    
    # Save updated mappings
    with open(mapping_file, 'w', encoding='utf-8') as f:
        json.dump(existing_mapping, f, indent=2, ensure_ascii=False)


Processar e padronizar a coluna brand no dataframe:


    Args:
        df (pd.DataFrame): Input dataframe com a coluna 'brands'
        
    Returns:
        pd.DataFrame: Dataframe com a coluna 'brands' padronizado

In [ ]:
def process_brands(df: pd.DataFrame) -> pd.DataFrame:
    
    # Load mapping
    mapping = get_brand_mapping()
    
    # Clean and standardize brands
    df['brands_clean'] = df['brands'].apply(clean_brand_text)
    
    # Apply mapping
    df['brands_final'] = df['brands_clean'].map(mapping).fillna(df['brands_clean'])
    
    return df

Para os dados não foram mapeados pelos os nomes padrão do primeiro dicionarios:

    Get list of brands that weren't mapped to any standard name.
    
    Args:
        df (pd.DataFrame): Processed dataframe with 'brands' column
        
    Returns:
        List[str]: List of unmapped brand names


In [ ]:
def get_unmapped_brands(df: pd.DataFrame) -> List[str]:

    mapping = get_brand_mapping()
    return sorted(df.loc[~df['brands'].isin(mapping.values()), 'brands'].unique())


In [41]:
def analyze_brand_distribution(df: pd.DataFrame) -> pd.Series:

    return df['brands'].value_counts() 


               id           code                   product_name  \
0   6111035000430  6111035000430                       SIDI ALI   
4   6111252421568  6111252421568                       اكوافينا   
6   3274080005003  3274080005003  CRISTALINE EAU DE SOURCE 1.5L   
23  6111242106949  6111242106949                           JBEN   
24  6111128000071  6111128000071                      AIN SAISS   

              brands                                         categories  \
0         [SIDI ALI]  BEVERAGES AND BEVERAGES PREPARATIONS,BEVERAGES...   
4   [PEPSI, PEPSICO]  BOISSONS ET PREPARATIONS DE BOISSONS,BOISSONS,...   
6       [CRISTALINE]  BOISSONS ET PREPARATIONS DE BOISSONS,BOISSONS,...   
23          [JAOUDA]  EN:DAIRIES, EN:FERMENTED FOODS, EN:FERMENTED M...   
24          [DANONE]  BOISSONS,EAUX,EAUX DE SOURCES,EAUX MINERALES,B...   

                                     ingredients_text  \
0                          une eau minérale naturelle   
4   ouverture et avant le : Voir

# Analises